# Packages

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import yfinance as yf
import yahoofinancials
from scipy.optimize import minimize
import seaborn as sn
from scipy.stats import anderson,kstest,shapiro, probplot

c:\Users\leona\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Masalah.  
Misalkan portofolio saham dengan komposisi saham TLKM, BBNI, dan ASII. Akan dicari nilai return dengan variansi minimal dan juga dicari komposisi dari masing-masing saham.  

In [2]:
stock = ['TSLA', 'DIS', 'NKE', 'AAPL', 'AXP', 'GS', 'GD', 'IBM', 'INTC', 'KO', 'PG', 'V', 'MA', 'BAC', 'NVDA', 'GOOGL', 'GOOG', 'FB', 'MSFT', 'AMZN',]
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()

In [3]:
mu = df.mean().values
C = df.cov().values
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [4]:
np.sum(np.round(x_,5))

0.9999899999999999

In [5]:
port_ = dict(zip(stock,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
port

{'IBM': 0.14139,
 'INTC': 0.19622,
 'KO': 0.1369,
 'PG': 0.31565,
 'BAC': 0.00279,
 'GOOGL': 0.06342,
 'GOOG': 0.05987,
 'MSFT': 0.06724,
 'AMZN': 0.01651}

In [6]:
start = 1e3 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget_

{'IBM': 141.39,
 'INTC': 196.22,
 'KO': 136.9,
 'PG': 315.65,
 'BAC': 2.79,
 'GOOGL': 63.42,
 'GOOG': 59.87,
 'MSFT': 67.24,
 'AMZN': 16.51}

In [7]:
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
budget

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Total Spread,141.39,196.22,136.9,315.65,2.79,63.42,59.87,67.24,16.51


In [8]:
expense = pd.DataFrame(columns=port.keys())
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
expense

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Date,,,,,,,,,
Closing 2022-05-20,12620.999908,4040.000153,5985.499954,13939.900208,3309.909821,212064.990234,213327.001953,24852.49939,210859.49707


In [9]:
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,IBM,INTC,KO,PG,BAC,GOOGL,GOOG,MSFT,AMZN
Closing 2022-05-20,12620.999908,4040.000153,5985.499954,13939.900208,3309.909821,212064.990234,213327.001953,24852.49939,210859.49707
Total Spread,141.390000,196.220000,136.900000,315.650000,2.790000,63.420000,59.870000,67.24000,16.51000
Total Lot,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
Total Spending,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000


In [10]:
finale.sum(axis = 1)

Closing 2022-05-20    701000.298691
Total Spread             999.990000
Total Lot                  0.000000
Total Spending             0.000000
dtype: float64

In [11]:
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

return : 0.5591821148432654 %
risk : 0.14885329350887289 %
Total Money : 0.0
Total Idle Money : 1000.0


In [12]:
finale.to_csv(str('USA '+finale.index[0].split(' ')[1]+'.csv'))

# Testing Profit

In [21]:
year = 2010
ds = str(year-1)+'-12-01'
de = str(year+2)+'-12-01'

stock = ['TSLA', 'DIS', 'NKE', 'AAPL', 'AXP', 'GS', 'GD', 'IBM', 'INTC', 'KO', 'PG', 'V', 'MA', 'BAC', 'NVDA', 'GOOGL', 'GOOG', 'FB', 'MSFT', 'AMZN',]
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()
df.dropna(axis = 1, inplace = True)

In [22]:
mu = df.mean().values
C = df.cov().values

def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [23]:
print(np.sum(np.round(x_,5)))
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

0.9999899999999999
return : 1.3841940733391596 %
risk : 0.057828309391543604 %
Total Money : 0.0
Total Idle Money : 1000.0


In [24]:
port_ = dict(zip(df.columns.values,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
start = 1e3 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
expense = pd.DataFrame(columns=port.keys())

bs = str(year+2)+'-12-01'
be = str(year+3)+'-01-01'
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,TSLA,DIS,NKE,IBM,INTC,KO,PG,V,MSFT,AMZN
Closing 2012-12-01,677.400017,4457.91626,2318.642807,12698.397064,1565.506649,2703.917694,5132.730103,3548.261642,2213.327408,25086.999512
Total Spread,15.060000,32.53000,27.510000,113.300000,0.360000,313.480000,339.930000,80.970000,2.090000,74.760000
Total Lot,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Total Spending,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
buy = finale.columns.values
bs = str(year+2)+'-12-01'
be = str(year+4)+'-01-01'
profit_test = pd.DataFrame()
for item in buy:
    temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
    temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
    profit_test['buy_'+item] = temp['buy_'+item].dropna()
    profit_test['sell_'+item] = temp['sell_'+item].dropna()
    profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
profit_test.dropna(inplace = True)
profit_test


,buy_TSLA,sell_TSLA,return_TSLA,buy_DIS,sell_DIS,return_DIS,buy_NKE,sell_NKE,return_NKE,buy_IBM,...,return_PG,buy_V,sell_V,return_V,buy_MSFT,sell_MSFT,return_MSFT,buy_AMZN,sell_AMZN,return_AMZN
Date,,,,,,,,,,,,,,,,,,,,,
2013-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-05-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-07-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-08-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
cols = profit_test.columns
finale_pt = pd.DataFrame()
tspend = 0
tprofit = 0 
for k in range (0,profit_test.shape[1], 3):
    tspend += profit_test[cols[k]]
    tprofit += profit_test[cols[k+2]]
finale_pt['Total Spending'] = tspend
finale_pt['Total Profit'] = tprofit
finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
finale_pt

,Total Spending,Total Profit,Total Return
Date,,,
2013-02-01,0.0,0.0,NaN
2013-03-01,0.0,0.0,NaN
2013-04-01,0.0,0.0,NaN
2013-05-01,0.0,0.0,NaN
2013-06-01,0.0,0.0,NaN
2013-07-01,0.0,0.0,NaN
2013-08-01,0.0,0.0,NaN
2013-09-01,0.0,0.0,NaN
2013-10-01,0.0,0.0,NaN


In [27]:
print(f'Keuntungan dalam satu tahun pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

Keuntungan dalam satu tahun pada tahun 2013 sebesar: nan %


# Loop

In [28]:
year = 2010
for n in range(8):
    year += 1
    ds = str(year-1)+'-12-01'
    de = str(year+2)+'-12-01'

    stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
            'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
    #stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
    df = pd.DataFrame(columns=stock)
    for i in stock :
        temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
        temp['log_return'].dropna(inplace = True)
        df[i] =temp['log_return'].dropna()
    df.dropna(axis = 1, inplace = True)

    mu = df.mean().values
    C = df.cov().values

    def objective(weights): 
        weights = np.array(weights)
        return weights.dot(C).dot(weights.T)
    # The constraints
    cons = (# The weights must sum up to one.
            {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
            # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
            #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

            )
    # Every stock can get any weight from 0 to 1
    bounds = tuple((0,1) for x in range(mu.shape[0]))
    # Initialize the weights with an even split
    # In out case each stock will have 10% at the beginning
    guess = [1./mu.shape[0] for x in range(mu.shape[0])]
    optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
    x_ = optimized_results.x
    print(np.sum(np.round(x_,5)))
    print(f'return :', x_.dot(mu)*100, '%')
    print(f'risk :', x_.dot(C).dot(x_)*100, '%')
    print(f'Total Money :',finale.sum(axis=1)[-1])
    print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])
    port_ = dict(zip(df.columns.values,np.round(x_,5)))
    port = dict()
    for (key,value) in port_.items():
        if value >0 : 
            port[key] = value
    start = 1e3 
    budget_ = dict()
    for (key,value) in port.items():
        if value >0 : 
            budget_[key] = value*start
    budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
    budget.rename(index={0: 'Total Spread'}, inplace= True)
    expense = pd.DataFrame(columns=port.keys())

    bs = str(year+2)+'-12-01'
    be = str(year+3)+'-01-01'

    for item in port.keys(): 
        temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
        expense[item] = temp['Close']*100 #1 lot
    expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
    qty = pd.concat([expense,budget])
    temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
    temp_.rename(index={0: 'Total Lot'}, inplace= True)
    qty = pd.concat([qty, temp_])
    spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
    finale = pd.concat([qty, spend])
    finale.rename(index={0: 'Total Spending'}, inplace= True)
    finale
    buy = finale.columns.values
    bs = str(year+2)+'-12-31'
    be = str(year+4)+'-01-01'
    profit_test = pd.DataFrame()
    for item in buy:
        temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
        temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
        profit_test['buy_'+item] = temp['buy_'+item].dropna()
        profit_test['sell_'+item] = temp['sell_'+item].dropna()
        profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
    profit_test.dropna(inplace = True)
    cols = profit_test.columns
    finale_pt = pd.DataFrame()
    tspend = 0
    tprofit = 0 
    for k in range (0,profit_test.shape[1], 3):
        tspend += profit_test[cols[k]]
        tprofit += profit_test[cols[k+2]]
    finale_pt['Total Spending'] = tspend
    finale_pt['Total Profit'] = tprofit
    finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
    finale_pt.to_csv('Profit Test USA' +str(year)+'.csv')
    print(f'Rata-rata satu tahun keuntungan perbulan pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

- ARTO.JK: Data doesn't exist for startDate = 1291136400, endDate = 1385830800
1.0
return : 0.5453092459352615 %
risk : 0.1339203958402126 %
Total Money : 0.0
Total Idle Money : 1000.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2014 sebesar: 1.210178657564042 %
- ARTO.JK: Data doesn't exist for startDate = 1322672400, endDate = 1417366800
1.0
return : 0.8671126185065634 %
risk : 0.0839737569608063 %
Total Money : 19474932.0
Total Idle Money : 525068.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2015 sebesar: -1.489487577357647 %
- ARTO.JK: Data doesn't exist for startDate = 1354294800, endDate = 1448902800
1.0
return : 0.4265608580841976 %
risk : 0.1098629577967746 %
Total Money : 19815568.0
Total Idle Money : 184432.0
Rata-rata satu tahun keuntungan perbulan pada tahun 2016 sebesar: 3.589437648645974 %
1.0000000000000002
return : 1.2310253853636206 %
risk : 0.08191463204174615 %
Total Money : 20149815.0
Total Idle Money : -149815.0
Rata-rata satu tahun keuntungan perb

KeyboardInterrupt: 